In [ ]:
import pandas as pd # type: ignore
from datetime import datetime # type: ignore
import glob
import numpy as np # type: ignore

#### Podatki

- df_eles_koledar
	- df_eles
	- df_prazniki
	- df_weather

- df_entsoe

- df_all_prices
	- ida
		- ida1
		- ida2
		- sipx
	- df_prices_volumes
		- df_prices
		- df_volumes

### df_eles_koledar

##### df_eles

In [2]:
df_eles = pd.read_csv('data/other/ELES.csv', sep=',', index_col=0)
df_eles.rename(columns={'Unnamed: 0.1': 'datum', 'PREDVIDEN PREVZEM': 'predviden_prevzem', 'DEJANSKI PREVZEM': 'dejanski_prevzem'}, inplace=True)
df_eles['datum'] = pd.to_datetime(df_eles.index)
df_eles.drop(columns=['hour'], inplace=True)
df_eles

,PREDVIDENA PROIZVODNJA,DEJANSKA PROIZVODNJA,predviden_prevzem,dejanski_prevzem,datum
2022-01-01 00:00:00,965.0,968.0,989.0,1044.0,2022-01-01 00:00:00
2022-01-01 01:00:00,959.0,949.0,953.0,1018.0,2022-01-01 01:00:00
2022-01-01 02:00:00,960.0,948.0,906.0,975.0,2022-01-01 02:00:00
2022-01-01 03:00:00,959.0,947.0,874.0,944.0,2022-01-01 03:00:00
2022-01-01 04:00:00,960.0,946.0,870.0,936.0,2022-01-01 04:00:00
...,...,...,...,...,...
2024-11-27 19:00:00,2022.0,NaN,1716.0,NaN,2024-11-27 19:00:00
2024-11-27 20:00:00,1871.0,NaN,1643.0,NaN,2024-11-27 20:00:00
2024-11-27 21:00:00,1715.0,NaN,1505.0,NaN,2024-11-27 21:00:00
2024-11-27 22:00:00,1520.0,NaN,1386.0,NaN,2024-11-27 22:00:00


##### df_prazniki

In [3]:
from prophet.make_holidays import make_holidays_df # type: ignore

df_prazniki = make_holidays_df(
    year_list=[2022 + i for i in range(3)], country='SI'
)
df_prazniki = df_prazniki.rename(columns={'ds': 'datum'})
df_prazniki.sort_values('datum', inplace=True)
df_prazniki['datum'] = pd.to_datetime(df_prazniki['datum'])

# Generate a new DataFrame with all dates between 2022-01-01 and 2024-10-23
df_all_dates = pd.DataFrame({'datum': pd.date_range('2022-01-01', '2024-12-31')})

# Merge the new DataFrame with df_prazniki, filling in missing values with 0
df_prazniki = pd.merge(df_all_dates, df_prazniki, on='datum', how='outer').fillna(0)

df_prazniki.tail()

,datum,holiday
1091,2024-12-27,0
1092,2024-12-28,0
1093,2024-12-29,0
1094,2024-12-30,0
1095,2024-12-31,0


##### df_weather

In [ ]:
df_forecast = pd.read_csv('data/weather/weather_fore_11-27.csv', sep=',')
# df_forecast = df_forecast.iloc[:, [0, 1, 9, 17]]
df_forecast['datum'] = pd.to_datetime(df_forecast['date']).apply(lambda t: t.replace(tzinfo=None))
df_forecast.drop(columns=['date'], inplace=True)
df_forecast

,temperature_2m,cloud_cover,direct_normal_irradiance,datum
0,2.374,100.0,0.0,2024-11-10 23:00:00
1,2.424,100.0,0.0,2024-11-11 00:00:00
2,2.424,100.0,0.0,2024-11-11 01:00:00
3,2.524,100.0,0.0,2024-11-11 02:00:00
4,2.474,100.0,0.0,2024-11-11 03:00:00
...,...,...,...,...
499,-1.385,0.0,0.0,2024-12-01 18:00:00
500,-2.135,24.0,0.0,2024-12-01 19:00:00
501,-2.635,48.0,0.0,2024-12-01 20:00:00
502,-2.985,72.0,0.0,2024-12-01 21:00:00


In [5]:
folder_path = 'data/weather'
df_weather = pd.concat([pd.read_csv(file) for file in glob.glob(folder_path + '/*.csv')], ignore_index=True)

df_weather = df_weather.iloc[:, [0,1,2,3]]
df_weather['datum'] = pd.to_datetime(df_weather['date']).apply(lambda t: t.replace(tzinfo=None))
df_weather.drop(columns=['date'], inplace=True)
df_weather

df_weather = pd.concat([df_weather, df_forecast])
df_weather

,temperature_2m,cloud_cover,direct_normal_irradiance,datum
0,5.130,0.0,0.0,2021-12-31 23:00:00
1,2.780,8.0,0.0,2022-01-01 00:00:00
2,3.580,14.0,0.0,2022-01-01 01:00:00
3,3.630,2.0,0.0,2022-01-01 02:00:00
4,3.980,23.0,0.0,2022-01-01 03:00:00
...,...,...,...,...
499,-1.385,0.0,0.0,2024-12-01 18:00:00
500,-2.135,24.0,0.0,2024-12-01 19:00:00
501,-2.635,48.0,0.0,2024-12-01 20:00:00
502,-2.985,72.0,0.0,2024-12-01 21:00:00


##### Merge df_eles_koledar

In [6]:
df_temp = pd.merge(df_eles, df_prazniki, on='datum', how='left')
df_eles_koledar = pd.merge(df_temp, df_weather, on='datum', how='right')
df_eles_koledar

,PREDVIDENA PROIZVODNJA,DEJANSKA PROIZVODNJA,predviden_prevzem,dejanski_prevzem,datum,holiday,temperature_2m,cloud_cover,direct_normal_irradiance
0,NaN,NaN,NaN,NaN,2021-12-31 23:00:00,NaN,5.130,0.0,0.0
1,965.0,968.0,989.0,1044.0,2022-01-01 00:00:00,New Year's Day,2.780,8.0,0.0
2,959.0,949.0,953.0,1018.0,2022-01-01 01:00:00,NaN,3.580,14.0,0.0
3,960.0,948.0,906.0,975.0,2022-01-01 02:00:00,NaN,3.630,2.0,0.0
4,959.0,947.0,874.0,944.0,2022-01-01 03:00:00,NaN,3.980,23.0,0.0
...,...,...,...,...,...,...,...,...,...
25771,NaN,NaN,NaN,NaN,2024-12-01 18:00:00,NaN,-1.385,0.0,0.0
25772,NaN,NaN,NaN,NaN,2024-12-01 19:00:00,NaN,-2.135,24.0,0.0
25773,NaN,NaN,NaN,NaN,2024-12-01 20:00:00,NaN,-2.635,48.0,0.0
25774,NaN,NaN,NaN,NaN,2024-12-01 21:00:00,NaN,-2.985,72.0,0.0


### df_entsoe

##### df_total_load

In [7]:
folder_path = 'data/entsoe/total_load_day_ahead'
df_total_load = pd.concat([pd.read_csv(file) for file in glob.glob(folder_path + '/*.csv')], ignore_index=True)

# Concatenate all the DataFrames together
df_total_load['datum'] = pd.to_datetime(df_total_load['Time (CET/CEST)'].str.split(pat=' - ', expand=True)[0],
										format='%d.%m.%Y %H:%M')
df_total_load = df_total_load.drop(columns=['Time (CET/CEST)'])
df_total_load

,Day-ahead Total Load Forecast [MW] - BZN|SI,Actual Total Load [MW] - BZN|SI,datum
0,1089.0,1150.0,2022-01-01 00:00:00
1,1056.0,1127.0,2022-01-01 01:00:00
2,1008.0,1083.0,2022-01-01 02:00:00
3,976.0,1053.0,2022-01-01 03:00:00
4,973.0,1047.0,2022-01-01 04:00:00
...,...,...,...
26302,-,-,2024-12-31 19:00:00
26303,-,-,2024-12-31 20:00:00
26304,-,-,2024-12-31 21:00:00
26305,-,-,2024-12-31 22:00:00


##### df_forecast_gen

In [8]:
folder_path = 'data/entsoe/generation_forecast'
df_forecast = pd.concat([pd.read_csv(file) for file in glob.glob(folder_path + '/*.csv')], ignore_index=True)
df_forecast['datum'] = pd.to_datetime(df_forecast['MTU'].str.split(pat=' - ', expand=True)[0],
										format='%d.%m.%Y %H:%M')
df_forecast = df_forecast.iloc[:, [1, -1]]
df_forecast

,Scheduled Generation [MW] (D) - BZN|SI,datum
0,1065.0,2022-01-01 00:00:00
1,1063.0,2022-01-01 01:00:00
2,1062.0,2022-01-01 02:00:00
3,1061.0,2022-01-01 03:00:00
4,1063.0,2022-01-01 04:00:00
...,...,...
26278,1553.0,2023-12-31 19:00:00
26279,1529.0,2023-12-31 20:00:00
26280,1544.0,2023-12-31 21:00:00
26281,1361.0,2023-12-31 22:00:00


##### df_forecast_solar

In [9]:
folder_path = 'data/entsoe/generation_forecast_solar'
df_forecast_solar = pd.concat([pd.read_csv(file) for file in glob.glob(folder_path + '/*.csv')], ignore_index=True)
df_forecast_solar['datum'] = pd.to_datetime(df_forecast_solar['MTU (CET/CEST)'].str.split(pat=' - ', expand=True)[0],
										format='%d.%m.%Y %H:%M')
df_forecast_solar = df_forecast_solar.iloc[:, [1, -1]]
df_forecast_solar

,Generation - Solar [MW] Day Ahead/ Slovenia (SI),datum
0,0.0,2022-01-01 00:00:00
1,0.0,2022-01-01 01:00:00
2,0.0,2022-01-01 02:00:00
3,0.0,2022-01-01 03:00:00
4,0.0,2022-01-01 04:00:00
...,...,...
26302,NaN,2024-12-31 19:00:00
26303,NaN,2024-12-31 20:00:00
26304,NaN,2024-12-31 21:00:00
26305,NaN,2024-12-31 22:00:00


##### df_entsoe - merge

In [10]:
df_entsoe = pd.merge(df_total_load, df_forecast_solar, on='datum', how='left')
df_entsoe = pd.merge(df_entsoe, df_forecast, on='datum', how='left')
df_entsoe

,Day-ahead Total Load Forecast [MW] - BZN|SI,Actual Total Load [MW] - BZN|SI,datum,Generation - Solar [MW] Day Ahead/ Slovenia (SI),Scheduled Generation [MW] (D) - BZN|SI
0,1089.0,1150.0,2022-01-01 00:00:00,0.0,1065.0
1,1089.0,1150.0,2022-01-01 00:00:00,0.0,1065.0
2,1056.0,1127.0,2022-01-01 01:00:00,0.0,1063.0
3,1056.0,1127.0,2022-01-01 01:00:00,0.0,1063.0
4,1008.0,1083.0,2022-01-01 02:00:00,0.0,1062.0
...,...,...,...,...,...
35083,-,-,2024-12-31 19:00:00,NaN,NaN
35084,-,-,2024-12-31 20:00:00,NaN,NaN
35085,-,-,2024-12-31 21:00:00,NaN,NaN
35086,-,-,2024-12-31 22:00:00,NaN,NaN


### df_all_prices

##### df_prices_volumes

In [11]:
df_prices = pd.read_csv('data/da/day_ahead_prices.csv', index_col=0)
df_prices.rename(columns={'Price': 'price'}, inplace=True)
df_volumes = pd.read_csv('data/da/day_ahead_volumes.csv', index_col=0)
df_volumes.rename(columns={'Price': 'volumes'}, inplace=True)

df_prices_volumes = pd.merge(df_prices, df_volumes, left_index=True, right_index=True, how='left')
df_prices_volumes.drop(columns=['Hour_q_x', 'Hour_q_y'], inplace=True)
df_prices_volumes['datum'] = df_prices_volumes.index
df_prices_volumes.reset_index(drop=True, inplace=True)
df_prices_volumes['datum'] = pd.to_datetime(df_prices_volumes['datum'])
df_prices_volumes['datum'] = df_prices_volumes['datum'].apply(lambda t: t.replace(tzinfo=None))
df_prices_volumes

C:\Users\patri\AppData\Local\Temp\ipykernel_3472\3606153352.py:10: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_prices_volumes['datum'] = pd.to_datetime(df_prices_volumes['datum'])


,price,volumes,datum
0,61.83783,942.2,2022-01-01 00:00:00
1,57.07848,1296.9,2022-01-02 00:00:00
2,20.89311,1130.2,2022-01-03 00:00:00
3,110.14028,1314.3,2022-01-04 00:00:00
4,109.85968,1348.5,2022-01-05 00:00:00
...,...,...,...
25500,85.09000,1383.6,2024-11-23 23:00:00
25501,93.02000,1475.0,2024-11-24 23:00:00
25502,108.12000,1414.7,2024-11-25 23:00:00
25503,120.38000,1595.0,2024-11-26 23:00:00


##### df_ida

In [12]:
it_prices = pd.read_csv('data/other/it_prices.csv')
it_prices[['left','right']] = it_prices['MTU (CET/CEST)'].str.split(' - ',expand=True)
it_prices = it_prices[['left', 'Day-ahead (EUR/MWh)']]
it_prices['datum'] = pd.to_datetime(it_prices['left'], format='%d/%m/%Y %H:%M:%S').apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))
it_prices.drop(columns='left', inplace=True)
it_prices.rename(columns={'Day-ahead (EUR/MWh)': 'ITprice'}, inplace=True)
it_prices.head()

,ITprice,datum
0,170.28,2022-01-01 00:00:00
1,155.72,2022-01-01 01:00:00
2,147.09,2022-01-01 02:00:00
3,91.00,2022-01-01 03:00:00
4,104.00,2022-01-01 04:00:00


In [13]:
hu_prices = pd.read_csv('data/other/hu_prices.csv')
hu_prices[['left','right']] = hu_prices['MTU (CET/CEST)'].str.split(' - ',expand=True)
hu_prices = hu_prices[['left', 'Day-ahead (EUR/MWh)']]
hu_prices['datum'] = pd.to_datetime(hu_prices['left'], format='%d/%m/%Y %H:%M:%S').apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))
hu_prices.drop(columns='left', inplace=True)
hu_prices.rename(columns={'Day-ahead (EUR/MWh)': 'HUprice'}, inplace=True)
hu_prices.tail()

,HUprice,datum
25483,185.68,2024-11-27 19:00:00
25484,183.02,2024-11-27 20:00:00
25485,158.59,2024-11-27 21:00:00
25486,141.64,2024-11-27 22:00:00
25487,131.56,2024-11-27 23:00:00


In [14]:
at_prices = pd.read_csv('data/other/AT_prices.csv')
at_prices[['left','right']] = at_prices['MTU (CET/CEST)'].str.split(' - ',expand=True)
at_prices = at_prices[['left', 'Day-ahead (EUR/MWh)', 'Sequence']]
at_prices['datum'] = pd.to_datetime(at_prices['left'], format='%d/%m/%Y %H:%M:%S').apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))
at_prices.drop(columns='left', inplace=True)
# at_prices = pd.pivot(at_prices, columns='Sequence', values='Day-ahead (EUR/MWh)')

at_prices['Sequence 1 Value'] = at_prices[at_prices['Sequence'] == 'Sequence 1']['Day-ahead (EUR/MWh)']
at_prices['Sequence 2 Value'] = at_prices[at_prices['Sequence'] == 'Sequence 2']['Day-ahead (EUR/MWh)']

at_prices = at_prices.pivot_table(
    index='datum', 
    columns='Sequence', 
    values='Day-ahead (EUR/MWh)', 
    aggfunc='first'
)

at_prices['datum'] = pd.to_datetime(at_prices.index)
# rename the columns
at_prices.rename(columns={'Sequence 1': 'AT1price', 'Sequence 2': 'AT2price'}, inplace=True)
at_prices.reset_index(drop=True, inplace=True)

at_prices

Sequence,AT1price,AT2price,datum
0,61.84,59.51,2022-01-01 00:00:00
1,61.84,57.10,2022-01-01 00:15:00
2,61.84,-39.66,2022-01-01 00:30:00
3,61.84,-40.03,2022-01-01 00:45:00
4,51.44,58.50,2022-01-01 01:00:00
...,...,...,...
101863,129.01,71.09,2024-11-27 22:45:00
101864,116.63,178.11,2024-11-27 23:00:00
101865,116.63,142.30,2024-11-27 23:15:00
101866,116.63,90.06,2024-11-27 23:30:00


In [15]:
# add IDA1, IDA2 and Sipx
ida1 = pd.read_csv('data/ida/ida1_prices.csv', parse_dates=['DeliveryDateTime'])
ida1.rename(columns={'Price': 'IDA1price'}, inplace=True)
ida1 = ida1[ida1['DeliveryDateTime'].isnull() == False]
ida1['DeliveryDateTime'] = ida1['DeliveryDateTime'].apply(lambda t: t.replace(tzinfo=None))

ida2 = pd.read_csv('data/ida/ida2_prices.csv', parse_dates=['DeliveryDateTime'])
ida2.rename(columns={'Price': 'IDA2price'}, inplace=True)
ida2 = ida2[ida2['DeliveryDateTime'].isnull() == False]
ida2['DeliveryDateTime'] = ida2['DeliveryDateTime'].apply(lambda t: t.replace(tzinfo=None))

sipx = pd.read_csv('data/other/sipx_prices.csv')
sipx = sipx[sipx['DeliveryDateTime'] > '2022-01-01']
sipx.rename(columns={'Price': 'SIPXprice'}, inplace=True)
sipx['DeliveryDateTime'] = pd.to_datetime(sipx['DeliveryDateTime']).apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))

# merge all three
idas = pd.merge(ida1, ida2, on='DeliveryDateTime', how='left').fillna(0)
idas['DeliveryDateTime'] = pd.to_datetime(idas['DeliveryDateTime']).apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))
df_ida = pd.merge(idas, sipx, on='DeliveryDateTime', how='right')

# clean a bit
df_ida['datum'] = pd.to_datetime(df_ida['DeliveryDateTime']).apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))
df_ida.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y', 'Unnamed: 0', 'DeliveryDateTime', 'Hour_q_x', 'Hour_q_y'], inplace=True)
df_ida.sort_values(by='datum', inplace=True)
df_ida

C:\Users\patri\AppData\Local\Temp\ipykernel_3472\841790980.py:15: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  sipx['DeliveryDateTime'] = pd.to_datetime(sipx['DeliveryDateTime']).apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))


,IDA1price,IDA2price,SIPXprice,datum
4244,50.10,75.0,61.83783,2022-01-01 00:00:00
4245,NaN,NaN,61.83783,2022-01-01 00:15:00
4246,NaN,NaN,61.83783,2022-01-01 00:30:00
4247,NaN,NaN,61.83783,2022-01-01 00:45:00
8492,50.10,75.0,51.44327,2022-01-01 01:00:00
...,...,...,...,...
93435,147.35,0.0,130.38000,2024-11-27 22:45:00
97680,146.23,0.0,120.00000,2024-11-27 23:00:00
97681,120.00,0.0,120.00000,2024-11-27 23:15:00
97682,129.69,0.0,120.00000,2024-11-27 23:30:00


In [16]:
df_ida = pd.merge(df_ida, hu_prices, on='datum', how='left')
df_ida['datum'] = pd.to_datetime(df_ida['datum'])
df_ida = pd.merge(df_ida, at_prices, on='datum', how='left')

it_prices['datum'] = pd.to_datetime(it_prices['datum'])
df_ida = pd.merge(df_ida, it_prices, on='datum', how='left')
df_ida['datum'] = df_ida['datum'].apply(lambda t: t.replace(tzinfo=None))

In [ ]:
df_all_prices = pd.merge(df_ida, df_prices_volumes, on='datum', how='left')
df_all_prices['price'] = df_all_prices['price'].ffill()
df_all_prices['volumes'] = df_all_prices['volumes'].ffill()
df_all_prices.sort_values(by='datum', inplace=True)

df_all_prices.tail()

,IDA1price,IDA2price,SIPXprice,datum,HUprice,AT1price,AT2price,ITprice,price,volumes
101911,147.35,0.0,130.38,2024-11-27 22:45:00,NaN,129.01,71.09,NaN,130.38,1694.2
101912,146.23,0.0,120.00,2024-11-27 23:00:00,131.56,116.63,178.11,NaN,120.00,1487.7
101913,120.00,0.0,120.00,2024-11-27 23:15:00,NaN,116.63,142.30,NaN,120.00,1487.7
101914,129.69,0.0,120.00,2024-11-27 23:30:00,NaN,116.63,90.06,NaN,120.00,1487.7
101915,129.56,0.0,120.00,2024-11-27 23:45:00,NaN,116.63,27.16,NaN,120.00,1487.7


### Merging the dataframes

In [25]:
df_temp = pd.merge(df_eles_koledar, df_entsoe, on='datum', how='left')
df = pd.merge(df_temp, df_all_prices, on='datum', how='right')
df.tail()

,PREDVIDENA PROIZVODNJA,DEJANSKA PROIZVODNJA,predviden_prevzem,dejanski_prevzem,datum,holiday,temperature_2m,cloud_cover,direct_normal_irradiance,Day-ahead Total Load Forecast [MW] - BZN|SI,...,Scheduled Generation [MW] (D) - BZN|SI,IDA1price,IDA2price,SIPXprice,HUprice,AT1price,AT2price,ITprice,price,volumes
110884,NaN,NaN,NaN,NaN,2024-11-27 22:45:00,NaN,NaN,NaN,NaN,NaN,...,NaN,147.35,0.0,130.38,NaN,129.01,71.09,NaN,130.38,1694.2
110885,1534.0,NaN,1302.0,NaN,2024-11-27 23:00:00,NaN,5.477,75.0,0.0,1413,...,NaN,146.23,0.0,120.00,131.56,116.63,178.11,NaN,120.00,1487.7
110886,NaN,NaN,NaN,NaN,2024-11-27 23:15:00,NaN,NaN,NaN,NaN,NaN,...,NaN,120.00,0.0,120.00,NaN,116.63,142.30,NaN,120.00,1487.7
110887,NaN,NaN,NaN,NaN,2024-11-27 23:30:00,NaN,NaN,NaN,NaN,NaN,...,NaN,129.69,0.0,120.00,NaN,116.63,90.06,NaN,120.00,1487.7
110888,NaN,NaN,NaN,NaN,2024-11-27 23:45:00,NaN,NaN,NaN,NaN,NaN,...,NaN,129.56,0.0,120.00,NaN,116.63,27.16,NaN,120.00,1487.7


In [ ]:
def clean_data(df):
	# Drop duplicate rows in column: 'datum'
	df = df.drop_duplicates(subset=['datum'])
	# Renames
	df = df.rename(columns={'datum': 'date', 
							'PREDVIDENA PROIZVODNJA': 'forecast_production',
							'DEJANSKA PROIZVODNJA': 'production',
							'predviden_prevzem': 'forecast_consumption',
							'dejanski_prevzem': 'consumption',
							'Day-ahead Total Load Forecast [MW] - BZN|SI': 'da_load',
							'Actual Total Load [MW] - BZN|SI': 'actual_load',
							'Generation - Solar  [MW] Day Ahead/ Slovenia (SI)': 'solar',
							'Scheduled Generation [MW] (D) - BZN|SI': 'scheduled_generation',
							'temperature_2m': 'T',
							'direct_normal_irradiance': 'irradiance',
						})
	
	df.index = df['date'].astype('datetime64[ns]')
	
	df = df[df['date'] < '2024-11-28 00:00:00']

	# choosing which values to interpolate from time series
	all_columns = df.columns.tolist()
	columns_to_drop = ['date', 'holiday']
	for column in columns_to_drop:
		all_columns.remove(column)
	
	for column in all_columns:
		df[column] = df[column].interpolate(method='time')
	
	# ffiling the other columns
	df = df.ffill()

	# removing leap year
	df = df[(df['date'].dt.day != 29) | (df['date'].dt.month != 2)]
	df = df.drop(df.index[0])
	# actual load is not relevant
	df.drop(columns=['actual_load'], inplace=True)
	return df

df = clean_data(df.copy())
df

C:\Users\patri\AppData\Local\Temp\ipykernel_3472\221114862.py:32: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df[column] = df[column].interpolate(method='time')


,forecast_production,production,forecast_consumption,consumption,date,holiday,T,cloud_cover,irradiance,da_load,...,scheduled_generation,IDA1price,IDA2price,SIPXprice,HUprice,AT1price,AT2price,ITprice,price,volumes
date,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:15:00,963.50,963.25,980.00,1037.50,2022-01-01 00:15:00,New Year's Day,2.9800,9.50,0.0,1089.0,...,1064.50,50.10,75.0,61.83783,56.7125,61.84,57.10,166.6400,61.83783,942.2
2022-01-01 00:30:00,962.00,958.50,971.00,1031.00,2022-01-01 00:30:00,New Year's Day,3.1800,11.00,0.0,1089.0,...,1064.00,50.10,75.0,61.83783,51.5850,61.84,-39.66,163.0000,61.83783,942.2
2022-01-01 00:45:00,960.50,953.75,962.00,1024.50,2022-01-01 00:45:00,New Year's Day,3.3800,12.50,0.0,1089.0,...,1063.50,50.10,75.0,61.83783,46.4575,61.84,-40.03,159.3600,61.83783,942.2
2022-01-01 01:00:00,959.00,949.00,953.00,1018.00,2022-01-01 01:00:00,New Year's Day,3.5800,14.00,0.0,1056.0,...,1063.00,50.10,75.0,51.44327,41.3300,51.44,58.50,155.7200,51.44327,1084.7
2022-01-01 01:15:00,959.25,948.75,941.25,1007.25,2022-01-01 01:15:00,New Year's Day,3.5925,11.00,0.0,1056.0,...,1062.75,50.10,80.4,51.44327,41.8025,51.44,46.10,153.5625,51.44327,1084.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-27 22:45:00,1530.50,2370.00,1323.00,1647.00,2024-11-27 22:45:00,0,5.6020,73.75,0.0,1498,...,1179.00,147.35,0.0,130.38000,134.0800,129.01,71.09,134.6700,130.38000,1694.2
2024-11-27 23:00:00,1534.00,2370.00,1302.00,1647.00,2024-11-27 23:00:00,0,5.4770,75.00,0.0,1413,...,1179.00,146.23,0.0,120.00000,131.5600,116.63,178.11,134.6700,120.00000,1487.7
2024-11-27 23:15:00,1534.00,2370.00,1302.00,1647.00,2024-11-27 23:15:00,0,5.4770,75.00,0.0,1413,...,1179.00,120.00,0.0,120.00000,131.5600,116.63,142.30,134.6700,120.00000,1487.7


In [23]:
print(df.columns)

Index(['forecast_production', 'production', 'forecast_consumption',
       'consumption', 'date', 'holiday', 'T', 'cloud_cover', 'irradiance',
       'da_load', 'solar', 'scheduled_generation', 'IDA1price', 'IDA2price',
       'SIPXprice', 'HUprice', 'AT1price', 'AT2price', 'ITprice', 'price',
       'volumes'],
      dtype='object')


In [ ]:
df.to_csv('data/dataset_task2.csv', index=False)